# Environment Setup

Run this notebook first to set up your environment for the RAG benchmarking framework. This will:
1. Install all required dependencies
2. Configure AWS access
3. Test imports and utilities
4. Create necessary directories

## 1. Install Dependencies

Run this cell to install all required packages. This only needs to be done once per environment.

In [ ]:
from utils.setup import install_requirements

success = install_requirements('requirements.txt')
print(f"[DEBUG] install_requirements returned: {success}")
if success:
    print("\n✅ All packages ready!")
else:
    print("\n❌ Package installation failed. Please check the errors above and try again.")

## 2. Configure AWS Access

Check AWS access and test permissions for each required service (S3, Bedrock, Neptune, OpenSearch).

In [ ]:
from utils.aws import check_aws_access

check_aws_access()

## 3. Test Imports

Verify that all required packages and utilities can be imported correctly.

In [ ]:
from utils.setup import test_imports

test_imports()

## 4. Create Directories

Set up necessary directories for results and data.

In [ ]:
from utils.setup import setup_directories

setup_directories()

## Setup Complete!

If all cells above show successful completion (✅), your environment is ready to use!

You can now proceed to:
1. Open notebooks in evaluation_pipelines/templates/
2. Create your own RAG implementations
3. Run evaluations and comparisons

All dependencies are installed in your current environment and will be available to all notebooks in this project.